<a href="https://colab.research.google.com/github/Bustagar/PROCESAMIENTO-DE-HABLA/blob/main/Desafio_3_ipynb_(Resuelto).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
cossim

array([0.1382319 , 0.1067036 , 0.23029327, ..., 0.12320753, 0.08765353,
       0.04415046])

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 2

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.



## Importación de librerías

In [155]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np



## Carga de datos

In [156]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [157]:
tfidfvect = TfidfVectorizer()

In [159]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# 'X_train' la podemos denominar como la matriz documento-término

In [160]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [163]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [164]:
# en 'y_train' guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

## Similaridad de docuementos

In [166]:
# Semilla para reproducibilidad
np.random.seed(42)

# Seleccionar 5 documentos aleatorios
random_docs = np.random.choice(X_train.shape[0], size=5, replace=False)

# Función auxiliar para imprimir separadores
def separator(char, length=80):
    print(char * length)

In [185]:
# Documentos elegidos aleatoriamente
random_docs

array([7492, 3546, 5582, 4793, 3813])

In [186]:
# Iterar sobre los documentos aleatorios
for idx in random_docs:
    separator('=', 80)
    print(f'DOCUMENTO BASE (índice {idx})')
    print(f'- Categoría: {newsgroups_train.target_names[y_train[idx]]}')
    print(f'- Preview: {newsgroups_train.data[idx][:500]} ...')
    separator('=', 80)

    # Calcular similitud de coseno (documento actual vs el resto)
    similarity = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los 5 documentos más similares al actual
    closest = np.argsort(similarity)[::-1][1:6]  # Ignorar el primer elemento (mismo doc)

    # Mostrar los documentos similares
    for i, sim_idx in enumerate(closest):
        print(f'SIMILAR #{i+1} (índice {sim_idx}, similitud {round(similarity[sim_idx], 2)})')
        print(f'- Categoría: {newsgroups_train.target_names[y_train[sim_idx]]}')
        print(f'- Preview: {newsgroups_train.data[sim_idx][:250]} ...')
        separator('_', 80) if i != 4 else print('\n\n')


DOCUMENTO BASE (índice 7492)
- Categoría: misc.forsale
- Preview: Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward"  ...
SIMILAR #1 (índice 2890, similitud 0.33)
- Categoría: sci.electronics
- Preview: 
To boorishly reply to myself, I found I did have the Instrutech information
already. The specs (to use the term loosely) are as follows:

A/D: 16 bit converter, with 14 bit accuracy to 100 kHz, 12 bit accuracy to
200 kHz. No specs for S/N, monotonic ...
________________________________________________________________________________
SIMILAR #2 (índice 3124, similitud 0.33)
- Categoría: sci.electronics
- Preview: 
OpenGL is a graphics programming library and as such is a great, portable
interface for the development of interactive 3D graphics applications. It
i

## Primer Documento:
  El documento Base es de la categoría "misc.forsale" se ven similitudes muy pequeñas en el similar #1 del documento de sci.electronics, esas similitudes pueden ser por terminos utilizados como por ejemplo Sistemas y no tienen relación entre su contenido y etiqueta

## Segundo Documento:

El documento base es de la categoría rec.sport.baseball se ven similitudes pequeñas en el similar #1 del documento de rec.sport.baseball, se aprecia que en su contenido hablan sobre software por lo que puede encontrar ahi la similitud y tambien comparten la misma etiqueta.



## Tercer documento:

Documento base sci.electronics VS Similar #1 rec.autos
La similitud es baja pero ambos menciones componenetes electronicos, por lo que tienen una similitud baja en relacion a los temas y etiqueta.


## Cuarto Documento:



Documento base comp.windows.x VS comp.windows.x, si bien la similitud es baja por el contenido  de cada documento, se aprecia que comparten la misma categoría y uso de lenguaje técnico.

## Quinto Documento:

Documento base soc.religion.christian VS talk.religion.misc tratan de dos temas diferentes, no teniendo relación en los temas pero si en alguno terminos el cual se pueden tomar como similitudes, dado tambien que las etiquetas son diferentes.

Muestra


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [187]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [197]:
# Definir valores de alpha para probar distintos niveles de suavización
alphas = np.round(np.linspace(0.05, 0.25, 5), 3)  # Redondear a 3 decimales
''' Utilizo valores pequeños porque permiten que el modelo confíe más en los datos observados, en cambio a los valores grandes hace que el modelo suavice demasiado y trate todas las palabras como si fueran comunes, afectando la clasificación.'''

' Utilizo valores pequeños porque permiten que el modelo confíe más en los datos observados, en cambio a los valores grandes hace que el modelo suavice demasiado y trate todas las palabras como si fueran comunes, afectando la clasificación.'

In [198]:
# Iterar sobre los valores de alpha y calcular el f1-score macro
for alpha in alphas:
    # Entrenar modelos con el alpha actual
    model_multinomial = MultinomialNB(alpha=alpha)
    model_complement = ComplementNB(alpha=alpha)

    model_multinomial.fit(X_train, y_train)
    model_complement.fit(X_train, y_train)

    # Predicciones
    y_pred_multinomial = model_multinomial.predict(X_test)
    y_pred_complement = model_complement.predict(X_test)

    # Calcular f1-score macro
    f1_multinomial = f1_score(y_test, y_pred_multinomial, average="macro")
    f1_complement = f1_score(y_test, y_pred_complement, average="macro")

    # Mostrar los resultados en el formato deseado
    print(f"- Alpha {alpha:.1f}\t MultinomialNB: {f1_multinomial:.16f}\t ComplementNB: {f1_complement:.16f}")

- Alpha 0.1	 MultinomialNB: 0.7075759537997842	 ComplementNB: 0.7483641766733619
- Alpha 0.1	 MultinomialNB: 0.6843541906952256	 ComplementNB: 0.7514057680620365
- Alpha 0.1	 MultinomialNB: 0.6619015810051616	 ComplementNB: 0.7546179560621163
- Alpha 0.2	 MultinomialNB: 0.6539516137690246	 ComplementNB: 0.7539233964304060
- Alpha 0.2	 MultinomialNB: 0.6456569046638646	 ComplementNB: 0.7518471930173904


Se observa que el modelo ComplementNB arroja mejores resultados que el modelo MultinominialNB, arrojando como mejor resultado ComplementNB: 0.7546179560621163 del tercer Alpha 0.1 ()



**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [199]:
# Transponer la matriz documento-término para obtener término-documento
X_terms = X_train.T

In [200]:
# Elegir las 5 palabras clave
palabras_elegidas = ["accessories", "game", "penalties", "players", "win"]

In [201]:
# Obtener índices de las palabras elegidas
indices_palabras = [list(palabras_elegidas).index(word) for word in palabras_elegidas]

In [202]:
# Transponer la matriz documento-término
X_terms = X_train.T

# Obtener nombres de las palabras en el vocabulario del vectorizador
feature_names = tfidfvect.get_feature_names_out()

# Elegir las 5 palabras clave y obtener sus índices dentro del vocabulario
palabras_elegidas = ["accessories", "game", "penalties", "players", "win"]
indices_palabras = [list(feature_names).index(word) for word in palabras_elegidas if word in feature_names]

# Estudiar similitud entre palabras
for idx, palabra in zip(indices_palabras, palabras_elegidas):
    print("="*50)
    print(f"\nPalabra: {palabra}")

    # Calcular similitud con todas las palabras
    similitudes_palabras = cosine_similarity(X_terms[idx], X_terms)[0]

    # Obtener las 5 palabras más similares (excluyendo la misma)
    palabras_similares = np.argsort(similitudes_palabras)[::-1][1:6]

    print("\nPalabras más similares:")
    for i, similaridad in zip(palabras_similares, similitudes_palabras[palabras_similares]):
        print(f"{i+1}. Palabra similar: {feature_names[i]} (similaridad: {similaridad:.3f})")


Palabra: accessories

Palabras más similares:
79590. Palabra similar: runing (similaridad: 0.506)
88925. Palabra similar: thule (similaridad: 0.482)
224. Palabra similar: 0096b11b (similaridad: 0.385)
821. Palabra similar: 08a283a0 (similaridad: 0.385)
22510. Palabra similar: bethesda (similaridad: 0.335)

Palabra: game

Palabras más similares:
42607. Palabra similar: games (similaridad: 0.246)
37955. Palabra similar: espn (similaridad: 0.224)
71815. Palabra similar: play (similaridad: 0.216)
91543. Palabra similar: ultima (similaridad: 0.207)
33264. Palabra similar: detroit (similaridad: 0.203)

Palabra: penalties

Palabras más similares:
25732. Palabra similar: capitalize (similaridad: 0.470)
67572. Palabra similar: ocured (similaridad: 0.451)
50119. Palabra similar: inlcludes (similaridad: 0.451)
45418. Palabra similar: halftime (similaridad: 0.451)
94532. Palabra similar: vitale (similaridad: 0.451)

Palabra: players

Palabras más similares:
56006. Palabra similar: league (similar

Las palabras similares obtenidas, fueron muy bajas y en donde la palabra con mayor similitu fue Palabra: penalties